# Large Language Models

Dass es sich bei Large Language Models (LLMs) nicht um Magie oder eine starke künstliche Intelligenz (oder gar einer Artificial Generel Intelligence bzw. AGI) sondern um reine Stastistik handelt, sollte mittlerweile allen bekannt sein.

Ein LLM macht doch nichts anderes, als an den vorhandenen Text ein weiteres Wort anzuhängen. Wort für Wort. Basierend auf Statistiken, die angeben, welches Wort die höchste Wahrscheinlichkeit hat, auf das oder die vorangegangenen Worte zu folgen. **Oder?!**

Ich möchte versuchen mich der Funktionsweise von Natural Language Processing (NLP) und LLMs anzunähern. Dabei werde ich über Begriffe, wie **Embeddings**, **Tokens**, **Attention** oder **Transformer** stolpern und versuchen zu erklären was hier passiert. Ich werde auch kurz auf **Rekurrente Neuronale Netze** eingehen, spielten sie doch zum Beispiel im Google Translator in Form des **Google Neural Machine Translation (GNMT)** Systems eine Rolle.

## Ein Wort nach dem anderem?

Language Models versuchen an den gegebenen Text sinnvoll Wörter anzuhängen. Unter sinnvoll versteht man, was man von einer Person zu schreiben erwarten würde, wenn diese Person Milliarden von Text, Büchern, Internetseiten usw. gelesen und sich dann gemerkt hätte, welches Wort am häufigsten auf den vorangegangenen Text oder das vorangegangene Wort folgte.

In [14]:
!pip install --upgrade llama-cpp-python langchain gpt4all llama-index sentence-transformers

     ---------------------------------------- 0.0/48.7 MB ? eta -:--:--
     ---------------------------------------- 0.3/48.7 MB 5.2 MB/s eta 0:00:10
     ---------------------------------------- 0.6/48.7 MB 6.2 MB/s eta 0:00:08
      --------------------------------------- 1.0/48.7 MB 7.0 MB/s eta 0:00:07
     - -------------------------------------- 1.3/48.7 MB 6.8 MB/s eta 0:00:08
     - -------------------------------------- 1.7/48.7 MB 7.1 MB/s eta 0:00:07
     - -------------------------------------- 2.0/48.7 MB 7.0 MB/s eta 0:00:07
     - -------------------------------------- 2.3/48.7 MB 7.1 MB/s eta 0:00:07
     -- ------------------------------------- 2.7/48.7 MB 7.1 MB/s eta 0:00:07
     -- ------------------------------------- 3.0/48.7 MB 7.1 MB/s eta 0:00:07
     -- ------------------------------------- 3.4/48.7 MB 7.1 MB/s eta 0:00:07
     --- ------------------------------------ 3.7/48.7 MB 7.1 MB/s eta 0:00:07
     --- ------------------------------------ 4.0/48.7 MB 7

ERROR: Could not install packages due to an OSError: [WinError 5] Zugriff verweigert: 'C:\\Users\\gutsc\\anaconda3\\envs\\tf_GPU\\Lib\\site-packages\\~lama_cpp\\llama.dll'
Consider using the `--user` option or check the permissions.



In [15]:
!huggingface-cli download TheBloke/Llama-2–7b-Chat-GGUF llama-2–7b-chat.Q5_K_S.gguf — local-dir . — local-dir-use-symlinks False

usage: huggingface-cli <command> [<args>]
huggingface-cli: error: argument {login,whoami,logout,repo,lfs-enable-largefiles,lfs-multipart-upload,scan-cache,delete-cache}: invalid choice: 'download' (choose from 'login', 'whoami', 'logout', 'repo', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'scan-cache', 'delete-cache')


In [19]:
!pip install llama-cpp-python==0.1.48

     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ------- -------------------------------- 0.2/1.1 MB 6.3 MB/s eta 0:00:01
     ------------------ --------------------- 0.5/1.1 MB 6.7 MB/s eta 0:00:01
     --------------------------- ------------ 0.8/1.1 MB 6.3 MB/s eta 0:00:01
     ---------------------------------------  1.1/1.1 MB 6.5 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 6.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.48-cp310-cp310-win_amd64.whl size=23401

  Using cached llama_cpp_python-0.1.48.tar.gz (1.1 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.48-cp310-cp310-win_amd64.whl size=234014 sha256=1c5f96193908d30b93a9d88c6f185d41d0b3f9d85ca63e884cf453323035c468
  Stored in directory: c:\users\gutsc\appdata\local\pip\cache\wheels\eb\06\0e\6ae7b299ed252075128644d31384cac683e9fd768a8538c6be
Successfully built llama-cpp-python
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama-cpp-python 0.2.75
    Can't uninstall 'llama-cpp-python'. No files were found to uninstall.


In [21]:
from llama_cpp import Llama

# Load the model
llama = Llama(model_path='/llama-2-7b-chat.Q5_K_S.gguf')

ValueError: Model path does not exist: /llama-2-7b-chat.Q5_K_S.gguf

In [24]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [ ]:
prompt = "A small step"
response = llama(prompt)
print(response["choices"][0]["text"])

## Rekurrente Neuronale Netze in Natural Language Processing

Im November 2016 revolutionierte Google die maschinelle Übersetzung mit der Einführung des GNMT-Systems (Google Neural Machine Translation). Dieses System nutzte eine innovative Encoder-Decoder-Architektur mit RNNs (LSTMs) und einer Aufmerksamkeitsmechanik. Durch den Einsatz von neuronalen Netzwerken verbesserte es die Übersetzungsgenauigkeit deutlich im Vergleich zum vorherigen statistischen Ansatz. Jedoch setzte Google 2017 ein noch effizienteres Modell ein - den Transformer. Dieser innovative Ansatz basierte ausschließlich auf Aufmerksamkeitsmechanismen, wie im hochgelobten Paper "Attention is All You Need" beschrieben. Vor der Einführung von GNMT verließ sich Google Translate auf herkömmliche statistische Methoden zur maschinellen Übersetzung. Diese bahnbrechenden Fortschritte haben die Präzision und Leistungsfähigkeit von Google Translate erheblich gesteigert.

Auch wenn RNNs nicht mehr State of the Art für NLP ist, lässt sich hier doch einiges veranschaulichen. Daher versuche ich im folgenden den IMDB Datensatz mit Hilfe von RNNs zu analysieren und Vorhersagen zu machen, ob eine Kritik z.B. positiv oder negativ ist.

### IMDB Datensatz


Der IMDB-Datensatz ist eine wertvolle Ressource für die natürliche Sprachverarbeitung (NLP). Er umfasst eine große Anzahl von Filmrezensionen, die jeweils mit Bewertungen versehen sind. Dieser Datensatz wird häufig in der Sentimentanalyse eingesetzt, um den sentimentalen Gehalt eines Textes zu klassifizieren, ob er positiv oder negativ ist. Mit Hilfe des IMDB-Datensatzes können Forscher und Entwickler NLP-Algorithmen trainieren und verbessern, um Texte besser zu verstehen und Bewertungen automatisch zu analysieren.

Es ist sozusagen das **Hallo Welt!** des NLP.

Im folgenden werde ich versuchen, dass Beispiel IMDB in Python umzusetzen und vorherzusagen, ob eine Kritik positiv oder negativ ist. Ich fange mit den wichtigsen Imports an.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Laden des IMDB-Datensatzes
vocab_size = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Vorbereiten der Daten
max_len = 200
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Erstellen des Modells
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 32, input_length=max_len),
    keras.layers.LSTM(64),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training des Modells
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

# Vorhersagen treffen
text = "This movie was amazing!"
sequence = imdb.get_word_index()
input_text = [sequence[word.lower()] if word.lower() in sequence else 0 for word in text.split()]
input_text = pad_sequences([input_text], maxlen=max_len)
prediction = model.predict(input_text)[0][0]

# Ausgabe der Vorhersage
if prediction >= 0.5:
    print("Positive Sentiment")
else:
    print("Negative Sentiment")